# Projeto Big Data & Data Mining

### Consumo de propagandas: impacto dos anúncios na decisão de compra dos consumidores

#### Imports

In [9]:
import pandas as pd 

#### Armazenamento

In [12]:
df = pd.read_csv("../data/Social_Media_Advertising.csv")
df.head()

,Campaign_ID,Target_Audience,Campaign_Goal,Duration,Channel_Used,Conversion_Rate,Acquisition_Cost,ROI,Location,Language,Clicks,Impressions,Engagement_Score,Customer_Segment,Date,Company
0,529013,Men 35-44,Product Launch,15 Days,Instagram,0.15,$500.00,5.790000,Las Vegas,Spanish,500,3000,7,Health,2022-02-25,Aura Align
1,275352,Women 45-60,Market Expansion,15 Days,Facebook,0.01,$500.00,7.210000,Los Angeles,French,500,3000,5,Home,2022-05-12,Hearth Harmony
2,692322,Men 45-60,Product Launch,15 Days,Instagram,0.08,$500.00,0.430000,Austin,Spanish,500,3000,9,Technology,2022-06-19,Cyber Circuit
3,675757,Men 25-34,Increase Sales,15 Days,Pinterest,0.03,$500.00,0.909824,Miami,Spanish,293,1937,1,Health,2022-09-08,Well Wish
4,535900,Men 45-60,Market Expansion,15 Days,Pinterest,0.13,$500.00,1.422828,Austin,French,293,1937,1,Home,2022-08-24,Hearth Harmony


#### Análise

In [13]:
df.shape

(300000, 16)

In [14]:
print("Primeiras 5 linhas do DataFrame: ")
print(df.head(5))

Primeiras 5 linhas do DataFrame: 
   Campaign_ID Target_Audience     Campaign_Goal Duration Channel_Used  \
0       529013       Men 35-44    Product Launch  15 Days    Instagram   
1       275352     Women 45-60  Market Expansion  15 Days     Facebook   
2       692322       Men 45-60    Product Launch  15 Days    Instagram   
3       675757       Men 25-34    Increase Sales  15 Days    Pinterest   
4       535900       Men 45-60  Market Expansion  15 Days    Pinterest   

   Conversion_Rate Acquisition_Cost       ROI     Location Language  Clicks  \
0             0.15          $500.00  5.790000    Las Vegas  Spanish     500   
1             0.01          $500.00  7.210000  Los Angeles   French     500   
2             0.08          $500.00  0.430000       Austin  Spanish     500   
3             0.03          $500.00  0.909824        Miami  Spanish     293   
4             0.13          $500.00  1.422828       Austin   French     293   

   Impressions  Engagement_Score Customer_Segm

In [15]:
print("Informações Gerais do DataFrame: ")
df.info()

Informações Gerais do DataFrame: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 16 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Campaign_ID       300000 non-null  int64  
 1   Target_Audience   300000 non-null  object 
 2   Campaign_Goal     300000 non-null  object 
 3   Duration          300000 non-null  object 
 4   Channel_Used      300000 non-null  object 
 5   Conversion_Rate   300000 non-null  float64
 6   Acquisition_Cost  300000 non-null  object 
 7   ROI               300000 non-null  float64
 8   Location          300000 non-null  object 
 9   Language          300000 non-null  object 
 10  Clicks            300000 non-null  int64  
 11  Impressions       300000 non-null  int64  
 12  Engagement_Score  300000 non-null  int64  
 13  Customer_Segment  300000 non-null  object 
 14  Date              300000 non-null  object 
 15  Company           300000 non-null 

In [16]:
df.describe()

,Campaign_ID,Conversion_Rate,ROI,Clicks,Impressions,Engagement_Score
count,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000
mean,550444.804487,0.080009,3.177691,18153.670370,56034.236387,4.369217
std,260252.586037,0.040563,2.461200,11027.023294,32583.136334,3.156492
min,100001.000000,0.010000,0.000000,293.000000,1937.000000,1.000000
25%,325003.500000,0.050000,0.930000,8821.000000,28362.000000,1.000000
50%,551164.500000,0.080000,2.670000,17230.000000,54098.000000,4.000000
75%,776284.500000,0.110000,5.330000,26808.000000,80925.250000,7.000000
max,999998.000000,0.150000,8.000000,40000.000000,120000.000000,10.000000


#### Tratamentos

In [ ]:
# Engenharia de Atributos: 
# Classificação de público-alvo (target audience)
print(df['Target_Audience'].unique())
print("\n--------Frequencia----------")
df['Target_Audience'].value_counts()

['Men 35-44' 'Women 45-60' 'Men 45-60' 'Men 25-34' 'Women 35-44'
 'All Ages' 'Women 25-34' 'Men 18-24' 'Women 18-24']

--------Frequencia----------


Target_Audience
Women 18-24    33593
Men 45-60      33491
Women 25-34    33482
All Ages       33447
Men 25-34      33346
Men 35-44      33204
Men 18-24      33181
Women 35-44    33142
Women 45-60    33114
Name: count, dtype: int64

In [18]:
# Separação de gênero em novas colunas e extação de faixa etária
df['Gender'] = df['Target_Audience'].str.extract(r'^(Men|Women|All Ages)', expand=False)
df[['Age_Min', 'Age_Max']] = df['Target_Audience'].str.extract(r'(\d+)-(\d+)', expand=True)

In [19]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 19 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Campaign_ID       300000 non-null  int64  
 1   Target_Audience   300000 non-null  object 
 2   Campaign_Goal     300000 non-null  object 
 3   Duration          300000 non-null  object 
 4   Channel_Used      300000 non-null  object 
 5   Conversion_Rate   300000 non-null  float64
 6   Acquisition_Cost  300000 non-null  object 
 7   ROI               300000 non-null  float64
 8   Location          300000 non-null  object 
 9   Language          300000 non-null  object 
 10  Clicks            300000 non-null  int64  
 11  Impressions       300000 non-null  int64  
 12  Engagement_Score  300000 non-null  int64  
 13  Customer_Segment  300000 non-null  object 
 14  Date              300000 non-null  object 
 15  Company           300000 non-null  object 
 16  Gender            30

In [20]:
#Conversão das idades em núméricos

df['Age_Max'] = pd.to_numeric(df['Age_Max'], errors='coerce').astype('Int64') # conversão direta para inteiro
df['Age_Min'] = pd.to_numeric(df['Age_Min'], errors='coerce').astype('Int64')
print(df['Age_Max'].head(5))

0    44
1    60
2    60
3    34
4    60
Name: Age_Max, dtype: Int64


In [21]:
print(df['Gender'].value_counts())

Gender
Women       133331
Men         133222
All Ages     33447
Name: count, dtype: int64


In [22]:
# Tratar esse All ages de alguma forma
# Se for gênero, posso deixar como undefined?
# Se for Undefined, como seriam as idades para conversão

In [23]:
#Conversão para inteiro || Esqueci que há NaN
#df['Age_Max'] = df['Age_Max'].astype(int)
#df['Age_Min'] = df['Age_Min'].astype(int)

In [24]:
# Duração de object para int
df['Duration'].unique()

array(['15 Days', '30 Days', '45 Days', '60 Days'], dtype=object)

In [25]:
df = df.rename(columns={'Duration': 'Duration_in_Days'})
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 19 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Campaign_ID       300000 non-null  int64  
 1   Target_Audience   300000 non-null  object 
 2   Campaign_Goal     300000 non-null  object 
 3   Duration_in_Days  300000 non-null  object 
 4   Channel_Used      300000 non-null  object 
 5   Conversion_Rate   300000 non-null  float64
 6   Acquisition_Cost  300000 non-null  object 
 7   ROI               300000 non-null  float64
 8   Location          300000 non-null  object 
 9   Language          300000 non-null  object 
 10  Clicks            300000 non-null  int64  
 11  Impressions       300000 non-null  int64  
 12  Engagement_Score  300000 non-null  int64  
 13  Customer_Segment  300000 non-null  object 
 14  Date              300000 non-null  object 
 15  Company           300000 non-null  object 
 16  Gender            30

In [26]:
df["Duration_in_Days"] = df["Duration_in_Days"].str.replace("Days", " ", regex=False)
df["Duration_in_Days"] = df["Duration_in_Days"].str.strip()

In [27]:
print(df['Duration_in_Days'].head(5))

0    15
1    15
2    15
3    15
4    15
Name: Duration_in_Days, dtype: object


In [28]:
df["Duration_in_Days"] = pd.to_numeric(df["Duration_in_Days"], errors="coerce").astype("Int64")

In [29]:
print(df['Duration_in_Days'].head(5))

0    15
1    15
2    15
3    15
4    15
Name: Duration_in_Days, dtype: Int64


In [30]:
# Data para datetime
df['Date'].head(3)

0    2022-02-25
1    2022-05-12
2    2022-06-19
Name: Date, dtype: object

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 19 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Campaign_ID       300000 non-null  int64  
 1   Target_Audience   300000 non-null  object 
 2   Campaign_Goal     300000 non-null  object 
 3   Duration_in_Days  300000 non-null  Int64  
 4   Channel_Used      300000 non-null  object 
 5   Conversion_Rate   300000 non-null  float64
 6   Acquisition_Cost  300000 non-null  object 
 7   ROI               300000 non-null  float64
 8   Location          300000 non-null  object 
 9   Language          300000 non-null  object 
 10  Clicks            300000 non-null  int64  
 11  Impressions       300000 non-null  int64  
 12  Engagement_Score  300000 non-null  int64  
 13  Customer_Segment  300000 non-null  object 
 14  Date              300000 non-null  object 
 15  Company           300000 non-null  object 
 16  Gender            30

In [32]:
df["Date"] = pd.to_datetime(df["Date"], format="%Y-%m-%d", errors="coerce")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 19 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   Campaign_ID       300000 non-null  int64         
 1   Target_Audience   300000 non-null  object        
 2   Campaign_Goal     300000 non-null  object        
 3   Duration_in_Days  300000 non-null  Int64         
 4   Channel_Used      300000 non-null  object        
 5   Conversion_Rate   300000 non-null  float64       
 6   Acquisition_Cost  300000 non-null  object        
 7   ROI               300000 non-null  float64       
 8   Location          300000 non-null  object        
 9   Language          300000 non-null  object        
 10  Clicks            300000 non-null  int64         
 11  Impressions       300000 non-null  int64         
 12  Engagement_Score  300000 non-null  int64         
 13  Customer_Segment  300000 non-null  object        
 14  Date

In [33]:
print(df['Date'])

0        2022-02-25
1        2022-05-12
2        2022-06-19
3        2022-09-08
4        2022-08-24
            ...    
299995   2022-10-17
299996   2022-02-25
299997   2022-10-02
299998   2022-01-24
299999   2022-07-18
Name: Date, Length: 300000, dtype: datetime64[ns]
